In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [2]:
vacc_file = 'vaccinationsdata.csv'
vacc_df = pd.read_csv(vacc_file)
vacc_df.head()

,Date,MMWR_week,Location,Distributed,Distributed_Janssen,Distributed_Moderna,Distributed_Pfizer,Distributed_Unk_Manuf,Dist_Per_100K,Distributed_Per_100k_12Plus,...,Administered_Dose1_Recip_5PlusPop_Pct,Series_Complete_5Plus,Series_Complete_5PlusPop_Pct,Administered_5Plus,Admin_Per_100k_5Plus,Distributed_Per_100k_5Plus,Series_Complete_Moderna_5Plus,Series_Complete_Pfizer_5Plus,Series_Complete_Janssen_5Plus,Series_Complete_Unk_Manuf_5Plus
0,01/21/2022,3,LA,7467120,318200,2957620,4191300,0,160625,190327,...,63.2,2390208.0,55.0,5865724.0,134927.0,171764.0,939747.0,1274979.0,174438.0,1044.0
1,01/21/2022,3,WA,16011165,734100,5776120,9500945,0,210261,246291,...,83.1,5298973.0,74.0,13190008.0,184259.0,223669.0,1853801.0,2999674.0,442148.0,3350.0
2,01/21/2022,3,CO,11371385,481900,4205300,6684185,0,197463,230374,...,81.6,3904471.0,72.0,9921557.0,182834.0,209551.0,1404341.0,2192272.0,305529.0,2329.0
3,01/21/2022,3,NV,5402390,255200,1851080,3296110,0,175393,206056,...,76.9,1786275.0,61.7,4525049.0,156328.0,186638.0,589391.0,1027647.0,169180.0,57.0
4,01/21/2022,3,HI,3148920,123800,1160860,1864260,0,222401,260003,...,89.7,1068153.0,80.3,2666120.0,200362.0,236645.0,357326.0,647324.0,63486.0,17.0


In [3]:
csv_file = "statesdata.csv"
state_data_df = pd.read_csv(csv_file)
state_data_df.head()

,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0


In [4]:
vacc_cols = ["Date", "Location", "Administered"]
vacc_chng = vacc_df[vacc_cols].copy()

vacc_chng = vacc_chng.rename(columns={"Location": "State"})

vacc_chng.head()

,Date,State,Administered
0,01/21/2022,LA,5866023
1,01/21/2022,WA,13190985
2,01/21/2022,CO,9922591
3,01/21/2022,NV,4525127
4,01/21/2022,HI,2666683


In [5]:
new_state_data_df = state_data_df[['date', 'state','cases','deaths']].copy()
new_state_data_df.head()

,date,state,cases,deaths
0,2020-01-21,Washington,1,0
1,2020-01-22,Washington,1,0
2,2020-01-23,Washington,1,0
3,2020-01-24,Illinois,1,0
4,2020-01-24,Washington,1,0


In [6]:
vacc_amnd = vacc_chng.loc[(vacc_chng['State'] == 'CA') |
                          (vacc_chng['State'] == 'NV') |
                          (vacc_chng['State'] == 'UT') |
                          (vacc_chng['State'] == 'AZ')]

vacc_amnd.head()

,Date,State,Administered
3,01/21/2022,NV,4525127
5,01/21/2022,CA,69706872
42,01/21/2022,UT,4699376
53,01/21/2022,AZ,10994834
79,01/20/2022,CA,69549938


In [7]:
SW = ['California', 'Nevada', 'Utah', 'Arizona']
new_state_data_df["region"] = np.where(new_state_data_df['state'].isin(SW), 'SW','Unknown')
aggregate_data_df = new_state_data_df.loc[new_state_data_df['region'] == 'SW']
aggregate_data_df.head()

,date,state,cases,deaths,region
5,2020-01-25,California,1,0,SW
8,2020-01-26,Arizona,1,0,SW
9,2020-01-26,California,2,0,SW
12,2020-01-27,Arizona,1,0,SW
13,2020-01-27,California,2,0,SW


In [8]:
vacc_amnd.loc[(vacc_amnd['State'] == 'CA') |
             (vacc_amnd['State'] == 'NV') |
             (vacc_amnd['State'] == 'UT') |
             (vacc_amnd['State'] == 'AZ'), ['State']] = 'SW'

vacc_amnd

/opt/anaconda3/envs/pythondata/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Date,State,Administered
3,01/21/2022,SW,4525127
5,01/21/2022,SW,69706872
42,01/21/2022,SW,4699376
53,01/21/2022,SW,10994834
79,01/20/2022,SW,69549938
...,...,...,...
26065,12/15/2020,SW,0
26079,12/14/2020,SW,0
26082,12/14/2020,SW,0
26118,12/14/2020,SW,0


In [9]:
sw_data_df = aggregate_data_df[['date', 'region','cases','deaths']].copy()
sw_data_df.head()

,date,region,cases,deaths
5,2020-01-25,SW,1,0
8,2020-01-26,SW,1,0
9,2020-01-26,SW,2,0
12,2020-01-27,SW,1,0
13,2020-01-27,SW,2,0


In [10]:
vacc_amnd['Date'] = pd.to_datetime(vacc_amnd['Date'])
vacc_date = vacc_amnd.groupby('Date').sum()
vacc_date.sort_values(by='Date')
vacc_date.reset_index()

/opt/anaconda3/envs/pythondata/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Date,Administered
0,2020-12-14,0
1,2020-12-15,0
2,2020-12-16,177
3,2020-12-17,1051
4,2020-12-18,2246
...,...,...
399,2022-01-17,89069347
400,2022-01-18,89170365
401,2022-01-19,89328007
402,2022-01-20,89737678


In [11]:
sw_data_df['date'] = pd.to_datetime(sw_data_df['date']).dt.date
sw_data_df.sort_values(by='date')
print(sw_data_df)

             date region    cases  deaths
5      2020-01-25     SW        1       0
8      2020-01-26     SW        1       0
9      2020-01-26     SW        2       0
12     2020-01-27     SW        1       0
13     2020-01-27     SW        2       0
...           ...    ...      ...     ...
38022  2022-01-20     SW   814547    4019
38033  2022-01-21     SW  1701949   25502
38035  2022-01-21     SW  7621774   78700
38060  2022-01-21     SW   614505    8736
38078  2022-01-21     SW   826149    4030

[2840 rows x 4 columns]


In [12]:
final_df = sw_data_df.groupby('date').sum()
final_df.sort_values(by='date')
final_df.reset_index()

,date,cases,deaths
0,2020-01-25,1,0
1,2020-01-26,3,0
2,2020-01-27,3,0
3,2020-01-28,3,0
4,2020-01-29,3,0
...,...,...,...
723,2022-01-17,10085863,115932
724,2022-01-18,10247594,116252
725,2022-01-19,10395170,116426
726,2022-01-20,10593531,116686


In [13]:
connection_string = "postgres:postgres@localhost:5433/COVID_merged"
engine = create_engine(f'postgresql://{connection_string}')

In [14]:
vacc_date.to_sql(name='vacc', con=engine)
final_df.to_sql(name='case_SW', con=engine)